# Gold Price Prediction - Modeling Phase

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [13]:
# Load Data
df = pd.read_csv("../data/filtered_data.csv", index_col="Date", parse_dates=True)
df = df.ffill()
df.head()

,Gold_Spot,Silver_Fututes,S&P_500,USD_Index,CPI,Treasury_Yield_10Y,Real_Interest_Rate,GPR,Unemployment
Date,,,,,,,,,
2006-01-31,549.86,9.850,1280.079956,99.4311,199.3,4.42,2.00,94.510017,4.7
2006-02-28,555.00,9.720,1280.660034,99.7695,199.4,4.57,2.02,95.858192,4.8
2006-03-31,557.09,11.480,1294.869995,100.5600,199.7,4.72,2.35,92.797623,4.7
2006-04-28,610.65,13.510,1310.609985,98.1412,200.7,4.99,2.39,89.015800,4.7
2006-05-31,675.39,12.398,1270.089966,97.7705,201.3,5.11,2.48,85.208778,4.6


In [16]:
X = df[["USD_Index"]]
y = df["Gold_Spot"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Prediction
y_pred = lin_reg.predict(X_test)

print("Coefficient: ", lin_reg.coef_)
print("Intercept: ", lin_reg.intercept_)

Coefficient:  [24.00260418]
Intercept:  -1113.9406008447324
